# RFMIP-CLEAR-SKY example (with NN) - version 2

*Last edited: 2024-11-15*

Compute clear-sky longwave fluxes. Based on git branch `nn_devel` from https://github.com/peterukk/

Instructions for building are at: `examples/rfmip-clear-sky/Readme.md`

## Notes

- The `rrtmgp_rfmip_{lw|sw}.F90` file has been changed to include `use_rrtmgp_nn = .true.`

- From the Readme.md: "The example programs have been modified to allow neural networks to be used by setting `use_rrtmgp_nn = .true.` (was `use_nn = .true.`) in `rrtmgp_rfmip_{lw|sw}.F90`. In addition, there is an option (`compare_flux =.true.`) to compare the output fluxes to benchmark line-by-line computations, alongside reference RRTMGP results which were produced in double precision."

- A copy of the original `rrtmgp_rfmip_{lw|sw}.F90` file was kept in `rrtmgp_rfmip_{lw|sw}-v1.F90`.

- The Makefile must be changed to include `FCFLAGS += $(OPTFLAGS) $(DEBUGFLAGS)`, both in RTE+RRTMGP libraries and in rrtmgp_rfmip_{lw|sw}.

## rrtmgp_rfmip_{lw|sw}.F90

ukk22test02/examples/rfmip-clear-sky/rrtmgp_rfmip_{lw|sw}.F90

<p style="font-size:1em;color:red;">The files needs to be manually modified to set the variable: `use_rrtmgp_nn = .true.`.</p>

## Build the RTE+RRTMGP libraries

In [23]:
%cd ukk22test02/build

[Errno 2] No such file or directory: 'ukk22test02/build'
/home/x/git/radnn/src/ukk22test02/examples/rfmip-clear-sky


In [24]:
%env FC=gfortran
%env FCFLAGS=-ffree-line-length-none -march=native -O3
%env NCHOME=/usr
%env NFHOME=/usr
%env BLASLIB=openblas

env: FC=gfortran
env: FCFLAGS=-ffree-line-length-none -march=native -O3
env: NCHOME=/usr
env: NFHOME=/usr
env: BLASLIB=openblas


In [25]:
! make clean

VAR="../../"
rm rrtmgp_rfmip_sw rrtmgp_rfmip_lw *.o *.mod *.optrpt
rm: cannot remove '*.optrpt': No such file or directory
make: [Makefile:162: clean] Error 1 (ignored)


In [26]:
! make DEBUGFLAGS=-pg

VAR="../../"
gfortran -ffree-line-length-none -march=native -O3 -DNC_NETCDF4  -pg -I../..//build -I/usr/include -c ../mo_simple_netcdf.F90 -fopenmp
gfortran -ffree-line-length-none -march=native -O3 -DNC_NETCDF4  -pg -I../..//build -I/usr/include -c easy_netcdf.F90 -fopenmp
gfortran -ffree-line-length-none -march=native -O3 -DNC_NETCDF4  -pg -I../..//build -I/usr/include -c mo_rfmip_io.F90 -fopenmp
gfortran -ffree-line-length-none -march=native -O3 -DNC_NETCDF4  -pg -I../..//build -I/usr/include -c ../mo_load_coefficients.F90 -fopenmp
gfortran -ffree-line-length-none -march=native -O3 -DNC_NETCDF4  -pg -I../..//build -I/usr/include -c rrtmgp_rfmip_lw.F90 -fopenmp
gfortran -ffree-line-length-none -march=native -O3 -DNC_NETCDF4  -pg -o rrtmgp_rfmip_lw rrtmgp_rfmip_lw.o mo_simple_netcdf.o easy_netcdf.o mo_rfmip_io.o mo_load_coefficients.o ../..//build/librte.a ../..//build/librrtmgp.a ../..//build/libneural.a -L../..//build -L/usr/lib -L/usr/lib -lrrtmgp -lrte -lneural -lnetcdff -lnetcdf 

## Build the examples

In [27]:
%cd ../examples/rfmip-clear-sky

[Errno 2] No such file or directory: '../examples/rfmip-clear-sky'
/home/x/git/radnn/src/ukk22test02/examples/rfmip-clear-sky


In [28]:
! make clean

VAR="../../"
rm rrtmgp_rfmip_sw rrtmgp_rfmip_lw *.o *.mod *.optrpt
rm: cannot remove '*.optrpt': No such file or directory
make: [Makefile:162: clean] Error 1 (ignored)


<br>
other options: OPTFLAGS=-Og DEBUGFLAGS="-g -pg"

In [29]:
! make DEBUGFLAGS=-pg

VAR="../../"
gfortran -ffree-line-length-none -march=native -O3 -DNC_NETCDF4  -pg -I../..//build -I/usr/include -c ../mo_simple_netcdf.F90 -fopenmp
gfortran -ffree-line-length-none -march=native -O3 -DNC_NETCDF4  -pg -I../..//build -I/usr/include -c easy_netcdf.F90 -fopenmp
gfortran -ffree-line-length-none -march=native -O3 -DNC_NETCDF4  -pg -I../..//build -I/usr/include -c mo_rfmip_io.F90 -fopenmp
gfortran -ffree-line-length-none -march=native -O3 -DNC_NETCDF4  -pg -I../..//build -I/usr/include -c ../mo_load_coefficients.F90 -fopenmp
gfortran -ffree-line-length-none -march=native -O3 -DNC_NETCDF4  -pg -I../..//build -I/usr/include -c rrtmgp_rfmip_lw.F90 -fopenmp
gfortran -ffree-line-length-none -march=native -O3 -DNC_NETCDF4  -pg -o rrtmgp_rfmip_lw rrtmgp_rfmip_lw.o mo_simple_netcdf.o easy_netcdf.o mo_rfmip_io.o mo_load_coefficients.o ../..//build/librte.a ../..//build/librrtmgp.a ../..//build/libneural.a -L../..//build -L/usr/lib -L/usr/lib -lrrtmgp -lrte -lneural -lnetcdff -lnetcdf 

## Run

### LW

In [30]:
! time ./rrtmgp_rfmip_lw \
    8 \
    multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc \
    ../../rrtmgp/data/rrtmgp-data-lw-g256-2018-12-04.nc \
    1 \
    1

 === using rrtmgp_nn, line 198 ===
 loading longwave absorption model from ../../neural/data/BEST_tau-lw-18-58-58.txt                                      
 loading Planck fraction model from ../../neural/data/BEST_pfrac-18-16-16.txt                                       
 Usage: rrtmgp_rfmip_lw [block_size] [rfmip_file] [k-distribution_file] [forcing_index (1,2,3)] [physics_index (1,2)] [optional gas optics input_output file]
 input file:multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc                                                                     
 ncol:         100
 nexp:          18
 nlay:          60
 Doing          225 blocks of size            8
 setting no2 to zero
 in total:           19  input gases
  shape play          60           8         225
 play sfc   103243.766     tlay sfc   311.876038    
 === using rrtmgp_nn, line 340 ===
 starting clear-sky longwave computations, using neural networks as RRTMGP kernel
 Elapsed time on everything    24.2220001 

In [31]:
! time ./rrtmgp_rfmip_lw \
    8 \
    multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc \
    ../../rrtmgp/data/rrtmgp-data-lw-g256-2018-12-04.nc \
    1 \
    1

 === using rrtmgp_nn, line 198 ===
 loading longwave absorption model from ../../neural/data/BEST_tau-lw-18-58-58.txt                                      
 loading Planck fraction model from ../../neural/data/BEST_pfrac-18-16-16.txt                                       
 Usage: rrtmgp_rfmip_lw [block_size] [rfmip_file] [k-distribution_file] [forcing_index (1,2,3)] [physics_index (1,2)] [optional gas optics input_output file]
 input file:multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc                                                                     
 ncol:         100
 nexp:          18
 nlay:          60
 Doing          225 blocks of size            8
 setting no2 to zero
 in total:           19  input gases
  shape play          60           8         225
 play sfc   103243.766     tlay sfc   311.876038    
 === using rrtmgp_nn, line 340 ===
 starting clear-sky longwave computations, using neural networks as RRTMGP kernel
 Elapsed time on everything    23.9160004 

In [32]:
! time ./rrtmgp_rfmip_lw \
    8 \
    multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc \
    ../../rrtmgp/data/rrtmgp-data-lw-g256-2018-12-04.nc \
    1 \
    1

 === using rrtmgp_nn, line 198 ===
 loading longwave absorption model from ../../neural/data/BEST_tau-lw-18-58-58.txt                                      
 loading Planck fraction model from ../../neural/data/BEST_pfrac-18-16-16.txt                                       
 Usage: rrtmgp_rfmip_lw [block_size] [rfmip_file] [k-distribution_file] [forcing_index (1,2,3)] [physics_index (1,2)] [optional gas optics input_output file]
 input file:multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc                                                                     
 ncol:         100
 nexp:          18
 nlay:          60
 Doing          225 blocks of size            8
 setting no2 to zero
 in total:           19  input gases
  shape play          60           8         225
 play sfc   103243.766     tlay sfc   311.876038    
 === using rrtmgp_nn, line 340 ===
 starting clear-sky longwave computations, using neural networks as RRTMGP kernel
 Elapsed time on everything    24.0319996 

In [33]:
real = [1.888, 1.811, 1.859]
sum(real)/len(real)

1.8526666666666667

### SW

In [34]:
! time ./rrtmgp_rfmip_sw \
    8 \
    multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc \
    ../../rrtmgp/data/rrtmgp-data-sw-g224-2018-12-04.nc \
    1 \
    1

 loading shortwave absorption model from ../../neural/data/BEST_tau-sw-abs-7-16-16-mae_2.txt                             
 loading rayleigh model from ../../neural/data/BEST_tau-sw-ray-7-16-16_2.txt                                 
 Usage: rrtmgp_rfmip_sw [block_size] [rfmip_file] [k-distribution_file] [forcing_index (1,2,3)] [optional gas optics input_output file]
 input file:multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc                                                                     
 ncol:         100
 nexp:          18
 nlay:          60
 ninputs:           7
 Doing          225 blocks of size            8
 Calculation uses RFMIP gases: h2o carbon_dioxide o3 nitrous_oxide carbon_monoxide methane oxygen nitrogen carbon_tetrachloride cfc11 cfc12 hcfc22 hfc143a hfc125 hfc23 hfc32 hfc134a cf4 no2 
 Calculation uses RFMIP gases: h2o co2 o3 n2o co ch4 o2 n2 ccl4 cfc11 cfc12 cfc22 hfc143a hfc125 hfc23 hfc32 hfc134a cf4 no2 
 setting no2 to zero
 starting clear-sky sh

In [35]:
! time ./rrtmgp_rfmip_sw \
    8 \
    multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc \
    ../../rrtmgp/data/rrtmgp-data-sw-g224-2018-12-04.nc \
    1 \
    1

 loading shortwave absorption model from ../../neural/data/BEST_tau-sw-abs-7-16-16-mae_2.txt                             
 loading rayleigh model from ../../neural/data/BEST_tau-sw-ray-7-16-16_2.txt                                 
 Usage: rrtmgp_rfmip_sw [block_size] [rfmip_file] [k-distribution_file] [forcing_index (1,2,3)] [optional gas optics input_output file]
 input file:multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc                                                                     
 ncol:         100
 nexp:          18
 nlay:          60
 ninputs:           7
 Doing          225 blocks of size            8
 Calculation uses RFMIP gases: h2o carbon_dioxide o3 nitrous_oxide carbon_monoxide methane oxygen nitrogen carbon_tetrachloride cfc11 cfc12 hcfc22 hfc143a hfc125 hfc23 hfc32 hfc134a cf4 no2 
 Calculation uses RFMIP gases: h2o co2 o3 n2o co ch4 o2 n2 ccl4 cfc11 cfc12 cfc22 hfc143a hfc125 hfc23 hfc32 hfc134a cf4 no2 
 setting no2 to zero
 starting clear-sky sh

In [36]:
! time ./rrtmgp_rfmip_sw \
    8 \
    multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc \
    ../../rrtmgp/data/rrtmgp-data-sw-g224-2018-12-04.nc \
    1 \
    1

 loading shortwave absorption model from ../../neural/data/BEST_tau-sw-abs-7-16-16-mae_2.txt                             
 loading rayleigh model from ../../neural/data/BEST_tau-sw-ray-7-16-16_2.txt                                 
 Usage: rrtmgp_rfmip_sw [block_size] [rfmip_file] [k-distribution_file] [forcing_index (1,2,3)] [optional gas optics input_output file]
 input file:multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc                                                                     
 ncol:         100
 nexp:          18
 nlay:          60
 ninputs:           7
 Doing          225 blocks of size            8
 Calculation uses RFMIP gases: h2o carbon_dioxide o3 nitrous_oxide carbon_monoxide methane oxygen nitrogen carbon_tetrachloride cfc11 cfc12 hcfc22 hfc143a hfc125 hfc23 hfc32 hfc134a cf4 no2 
 Calculation uses RFMIP gases: h2o co2 o3 n2o co ch4 o2 n2 ccl4 cfc11 cfc12 cfc22 hfc143a hfc125 hfc23 hfc32 hfc134a cf4 no2 
 setting no2 to zero
 starting clear-sky sh

## Gprof

### LW

In [37]:
! gprof ./rrtmgp_rfmip_lw

Flat profile:

Each sample counts as 0.01 seconds.
  %   cumulative   self              self     total           
 time   seconds   seconds    calls  ms/call  ms/call  name    
 50.00      0.01     0.01     3906     0.00     0.00  __mo_gas_concentrations_MOD_set_vmr_2d
 50.00      0.02     0.01                             __mo_rte_solver_kernels_MOD_sw_solver_2stream
  0.00      0.02     0.00    83233     0.00     0.00  lw_solver_noscat_GaussQuad
  0.00      0.02     0.00     2702     0.00     0.00  __mo_rte_lw_MOD_rte_lw
  0.00      0.02     0.00     1800     0.00     0.00  sw_layer_props_sources_2str
  0.00      0.02     0.00     1719     0.00     0.00  zero_array_4D
  0.00      0.02     0.00     1358     0.00     0.00  __mo_optical_props_MOD___copy_mo_optical_props_Ty_optical_props_2str
  0.00      0.02     0.00     1125     0.00     0.00  __mo_gas_concentrations_MOD___final_mo_gas_concentrations_Conc_field
  0.00      0.02     0.00      900     0.00     0.00  __mod_random_MOD_randn

### SW

In [38]:
! gprof ./rrtmgp_rfmip_sw    # gprof uses only the executable name (without parameters)

Flat profile:

Each sample counts as 0.01 seconds.
  %   cumulative   self              self     total           
 time   seconds   seconds    calls  ms/call  ms/call  name    
 50.00      0.01     0.01      225     0.04     0.04  __mo_gas_concentrations_MOD_init
 50.00      0.02     0.01      225     0.04     0.04  __mo_rte_solver_kernels_MOD_sw_solver_2stream
  0.00      0.02     0.00    79987     0.00     0.00  __mo_rrtmgp_util_string_MOD_lower_case
  0.00      0.02     0.00     5625     0.00     0.00  __mo_gas_concentrations_MOD_find_gas
  0.00      0.02     0.00     3681     0.00     0.00  __mo_gas_concentrations_MOD_set_vmr_scalar
  0.00      0.02     0.00     1800     0.00     0.00  adding
  0.00      0.02     0.00     1719     0.00     0.00  __mo_rte_util_array_MOD_any_vals_outside_2d
  0.00      0.02     0.00     1575     0.00     0.00  __mo_rte_util_array_MOD_extents_are_2d
  0.00      0.02     0.00     1125     0.00     0.00  __mo_gas_concentrations_MOD_get_conc_dims_and_iga